In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [3]:
from static_grader import grader

# Program Flow exercises

The objective of these exercises is to develop your ability to use iteration and conditional logic to build reusable functions. We will be extending our `get_primes` example from the [Program Flow notebook](../PY_ProgramFlow.ipynb) for testing whether much larger numbers are prime. Large primes are useful for encryption. It is too slow to test every possible factor of a large number to determine if it is prime, so we will take a different approach.

## Exercise 1: `mersenne_numbers`

A Mersenne number is any number that can be written as $2^p - 1$ for some $p$. For example, 3 is a Mersenne number ($2^2 - 1$) as is 31 ($2^5 - 1$). We will see later on that it is easy to test if Mersenne numbers are prime.

Write a function that accepts an exponent $p$ and returns the corresponding Mersenne number.

In [4]:
def mersenne_number(p):
    mersnenne_num = 2**p-1
    return mersenne_num

Mersenne numbers can only be prime if their exponent, $p$, is prime. Make a list of the Mersenne numbers for all primes $p$ between 3 and 65 (there should be 17 of them).

Hint: It may be useful to modify the `is_prime` and `get_primes` functions from [the Program Flow notebook](PY_ProgramFlow.ipynb) for use in this problem.

In [5]:
# we can make a list like this
my_list = [0, 1, 2]
print(my_list)

[0, 1, 2]


In [6]:
# we can also make an empty list and add items to it
another_list = []
print(another_list)

for item in my_list:
    another_list.append(item)

print(another_list)

[]
[0, 1, 2]


In [55]:
def is_prime(number):
    if number <=1:
        return False
    for factor in range(2, number):
        if number%factor ==0:
            return False
        
    return True

def get_primes(n_start, n_end):
    list_primes = []  # list with Mersenne prime numbers
    for p in range(n_start, n_end+1):
        if is_prime(p):
            num = 2**p-1
            list_primes.append(num)
            
    return list_primes

The next cell shows a dummy solution, a list of 17 sevens. Alter the next cell to make use of the functions you've defined above to create the appropriate list of Mersenne numbers.

In [56]:
mersennes = get_primes(3,65) #mersenne primes

In [57]:
print(mersennes)

[7, 31, 127, 2047, 8191, 131071, 524287, 8388607, 536870911, 2147483647, 137438953471, 2199023255551, 8796093022207, 140737488355327, 9007199254740991, 576460752303423487, 2305843009213693951]


In [58]:
grader.score.ip__mersenne_numbers(mersennes)

Your score:  1.0


## Exercise 2: `lucas_lehmer`

We can test if a Mersenne number is prime using the [Lucas-Lehmer test](https://en.wikipedia.org/wiki/Lucas%E2%80%93Lehmer_primality_test). First let's write a function that generates the sequence used in the test. Given a Mersenne number with exponent $p$, the sequence can be defined as

$$ n_0 = 4 $$
$$ n_i = (n_{i-1}^2 - 2) mod (2^p - 1) $$

Write a function that accepts the exponent $p$ of a Mersenne number and returns the Lucas-Lehmer sequence up to $i = p - 2$ (inclusive). Remember that the [modulo operation](https://en.wikipedia.org/wiki/Modulo_operation) is implemented in Python as `%`.

In [11]:
def lucas_lehmer(p):  
    mer_list=[4]      # list with calculated elements, first element is n0=4
    for i in range(1, p-1):
        val_calc=(mer_list[i-1]**2-2)%(2**p-1) # recursion with one term
        mer_list.append(val_calc)
        
    return mer_list

In [12]:
print(lucas_lehmer(17))

[4, 14, 194, 37634, 95799, 119121, 66179, 53645, 122218, 126220, 70490, 69559, 99585, 78221, 130559, 0]


Use your function to calculate the Lucas-Lehmer series for $p = 17$ and pass the result to the grader.

In [14]:
ll_result = lucas_lehmer(17)

grader.score.ip__lucas_lehmer(ll_result)

Your score:  1.0


## Exercise 3: `mersenne_primes`

For a given Mersenne number with exponent $p$, the number is prime if the Lucas-Lehmer series is 0 at position $p-2$. Write a function that tests if a Mersenne number with exponent $p$ is prime. Test if the Mersenne numbers with prime $p$ between 3 and 65 (i.e. 3, 5, 7, ..., 61) are prime. Your final answer should be a list of tuples consisting of `(Mersenne exponent, 0)` (or `1`) for each Mersenne number you test, where `0` and `1` are replacements for `False` and `True` respectively.

_HINT: The `zip` function is useful for combining two lists into a list of tuples_

In [15]:
def ll_prime(p):
    l1_mer_exp =[]
    l2_bool=[]
    tuples_list=[]
    
    
    for i in [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 43, 47, 49, 53, 59, 61]:
        test_list = lucas_lehmer(i)
        if test_list[-1]==0:
            l1_mer_exp.append(i)
            l2_bool.append(1)
        else:
            l1_mer_exp.append(i)
            l2_bool.append(0)
    
    tuples_list = list(zip(l1_mer_exp, l2_bool))
        
    return tuples_list
    

In [16]:
mersenne_primes = ll_prime(65) 
grader.score.ip__mersenne_primes(mersenne_primes)

Your score:  0.9411764705882354


## Exercise 4: Optimize `is_prime`

You might have noticed that the primality check `is_prime` we developed before is somewhat slow for large numbers. This is because we are doing a ton of extra work checking every possible factor of the tested number. We will use two optimizations to make a `is_prime_fast` function.

The first optimization takes advantage of the fact that two is the only even prime.  Thus we can check if a number is even and as long as its greater than 2, we know that it is not prime.

Our second optimization takes advantage of the fact that when checking factors, we only need to check odd factors up to the square root of a number.  Consider a number $n$ decomposed into factors $n=ab$.  There are two cases, either $n$ is prime and without loss of generality, $a=n, b=1$ or $n$ is not prime and $a,b \neq n,1$.  In this case, if $a > \sqrt{n}$, then $b<\sqrt{n}$.  So we only need to check all possible values of $b$ and we get the values of $a$ for free!  This means that even the simple method of checking factors will increase in complexity as a square root compared to the size of the number instead of linearly.

Lets write the function to do this and check the speed!  `is_prime_fast` will take a number and return whether or not it is prime.

You will see the functions followed by a cell with an `assert` statement.  These cells should run and produce no output, if they produce an error, then your function needs to be modified.  Do not modify the assert statements, they are exactly as they should be!

In [21]:
import math
def is_prime_fast(number):
    if (number>2)and(number%2==0):
        return False
    else:
        for factor in range(3, int(math.sqrt(number))+1):
            if (factor%2!=0)and(number%factor==0):
                return False
        
    return True

In [18]:
print(is_prime_fast(125))

False


Run the following cell to make sure it finds the same primes as the original function.

In [59]:
for n in range(10000):
    assert is_prime(n) == is_prime_fast(n)

AssertionError: 

Now lets check the timing, here we will use the `%%timeit` magic which will time the execution of a particular cell.

In [21]:
%%timeit
is_prime(67867967)

5.15 s ± 129 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
is_prime_fast(67867967)

713 µs ± 7.81 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Now return a function which will find all prime numbers up to and including $n$. Submit this function to the grader.

In [23]:
def get_primes_fast(n):
    primes_list = []
    for i in range(2,n+1):
        if is_prime_fast(i)==True:
            primes_list.append(i)
            
    return primes_list

In [25]:
grader.score.ip__is_prime_fast(get_primes_fast)

Your score:  1.0


## Exercise 5: sieve

In this problem we will develop an even faster method which is known as the Sieve of Eratosthenes (although it will be more expensive in terms of memory). The Sieve of Eratosthenes is an example of dynamic programming, where the general idea is to not redo computations we have already done (read more about it [here](https://en.wikipedia.org/wiki/Dynamic_programming)).  We will break this sieve down into several small functions. 

Our submission will be a list of all prime numbers less than 2000.

The method works as follows (see [here](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) for more details)

1. Generate a list of all numbers between 0 and N; mark the numbers 0 and 1 to be not prime
2. Starting with $p=2$ (the first prime) mark all numbers of the form $np$ where $n>1$ and $np <= N$ to be not prime (they can't be prime since they are multiples of 2!)
3. Find the smallest number greater than $p$ which is not marked and set that equal to $p$, then go back to step 2.  Stop if there is no unmarked number greater than $p$ and less than $N+1$

We will break this up into a few functions, our general strategy will be to use a Python `list` as our container although we could use other data structures.  The index of this list will represent numbers.

We have implemented a `sieve` function which will find all the prime numbers up to $n$.  You will need to implement the functions which it calls.  They are as follows

* `list_true` Make a list of true values of length $n+1$ where the first two values are false (this corresponds with step 1 of the algorithm above)
* `mark_false` takes a list of booleans and a number $p$.  Mark all elements $2p,3p,...n$ false (this corresponds with step 2 of the algorithm above)
* `find_next` Find the smallest `True` element in a list which is greater than some $p$ (has index greater than $p$ (this corresponds with step 3 of the algorithm above)
* `prime_from_list` Return indices of True values

Remember that python lists are zero indexed. We have provided assertions below to help you assess whether your functions are functioning properly.

In [66]:
def list_true(n):
    list_true=[False, False]+[True for i in range(2, n+1)]
    return list_true

In [67]:
print(list_true(20))

[False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [68]:
assert len(list_true(20)) == 21
assert list_true(20)[0] is False
assert list_true(20)[1] is False

Now we want to write a function which takes a list of elements and a number $p$ and marks elements false which are in the range $2p,3p ... N$.

In [29]:
def mark_false(bool_list, p):
    bool_list[0]=False
    bool_list[1]=False
    for n in range(2, int(len(bool_list)/p)+1): 
        bool_list[n*p]=False
    return bool_list

In [30]:
assert mark_false(list_true(6), 2) == [False, False, True, True, False, True, False]

Now lets write a `find_next` function which returns the smallest element in a list which is not false and is greater than $p$.

In [31]:
def find_next(bool_list, p):
    for i in range(len(bool_list)): 
        if (bool_list[i]!=False) and (i>p):
            smallest_elem = i
        else:
            smallest_elem = None
    return smallest_elem

In [32]:
assert find_next([True, True, True, True], 2) == 3
assert find_next([True, True, True, False], 2) is None

Now given a list of `True` and `False`, return the index of the true values.

In [33]:
def prime_from_list(bool_list):
    true_list = []
    for i in range (len(bool_list)):
        if bool_list[i]==True:
            true_list.append(i)
    return true_list

In [34]:
assert prime_from_list([False, False, True, True, False]) ==  [2, 3]

In [71]:
def sieve(n):
    bool_list = list_true(n)
    p = 2
    while p is not None:
        bool_list = mark_false(bool_list, p)
        p = find_next(bool_list, p)
    return prime_from_list(bool_list)

In [72]:
print(sieve(1000))

[2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223, 225, 227, 229, 231, 233, 235, 237, 239, 241, 243, 245, 247, 249, 251, 253, 255, 257, 259, 261, 263, 265, 267, 269, 271, 273, 275, 277, 279, 281, 283, 285, 287, 289, 291, 293, 295, 297, 299, 301, 303, 305, 307, 309, 311, 313, 315, 317, 319, 321, 323, 325, 327, 329, 331, 333, 335, 337, 339, 341, 343, 345, 347, 349, 351, 353, 355, 357, 359, 361, 363, 365, 367, 369, 371, 373, 375, 377, 379, 381, 383, 385, 387, 389, 391, 393, 395, 397, 399, 401, 403, 405, 407, 409, 411, 413, 415, 417, 419, 421,

In [73]:
print(get_primes(0, 1000))

[3, 7, 31, 127, 2047, 8191, 131071, 524287, 8388607, 536870911, 2147483647, 137438953471, 2199023255551, 8796093022207, 140737488355327, 9007199254740991, 576460752303423487, 2305843009213693951, 147573952589676412927, 2361183241434822606847, 9444732965739290427391, 604462909807314587353087, 9671406556917033397649407, 618970019642690137449562111, 158456325028528675187087900671, 2535301200456458802993406410751, 10141204801825835211973625643007, 162259276829213363391578010288127, 649037107316853453566312041152511, 10384593717069655257060992658440191, 170141183460469231731687303715884105727, ...]


In [69]:
assert sieve(1000) == get_primes(0, 1000) #sieve(1000) returns prime number within (0, 1000)and 
                                          #get_primes(0, 1000) returns mersenne primes; the two  
                                          #list are totally different (see above); 
                                          #therefore, the assert test between the two seems 
                                          #to be wrong...

AssertionError: 

In [37]:
%%timeit 
sieve(1000)

236 µs ± 1.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [38]:
%%timeit 
get_primes(0, 1000)

5.08 ms ± 60.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [76]:
grader.score.ip__eratosthenes(sieve)

[2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223, 225, 227, 229, 231, 233, 235, 237, 239, 241, 243, 245, 247, 249, 251, 253, 255, 257, 259, 261, 263, 265, 267, 269, 271, 273, 275, 277, 279, 281, 283, 285, 287, 289, 291, 293, 295, 297, 299, 301, 303, 305, 307, 309, 311, 313, 315, 317, 319, 321, 323, 325, 327, 329, 331, 333, 335, 337, 339, 341, 343, 345, 347, 349, 351, 353, 355, 357, 359, 361, 363, 365, 367, 369, 371, 373, 375, 377, 379, 381, 383, 385, 387, 389, 391, 393, 395, 397, 399, 401, 403, 405, 407, 409, 411, 413, 415, 417, 419, 421,

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*